In [1]:
''' from google.colab import drive
drive.mount('/content/drive')
 '''

# clone the github repo
#!git clone https://github.com/OpenGVLab/InternVL.git

# cd into the cloned repo
%cd /content/drive/MyDrive/FYP/InternVL

# install all the dependencies
!pip install -r requirements.txt
!pip install -r requirements/clip_benchmark.txt
!pip install fastapi uvicorn nest-asyncio pyngrok transformers torch decord pillow
!pip install protobuf==3.20.*
!pip install python-multipart

Mounted at /content/drive
/content/drive/MyDrive/FYP/InternVL
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is tak

  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl.metadata (422 bytes)
  Using cached starlette-0.41.3-py3-none-any.whl.metadata (6.0 kB)
Using cached fastapi-0.115.5-py3-none-any.whl (94 kB)
Using cached uvicorn-0.32.1-py3-none-any.whl (63 kB)
Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl (13.6 MB)
Using cached starlette-0.41.3-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.2

  Using cached python_multipart-0.0.17-py3-none-any.whl.metadata (1.8 kB)
Using cached python_multipart-0.0.17-py3-none-any.whl (24 kB)


In [ ]:
#install the pretrained model (InternVL-2 4B) weights from hugging face

!pip install -U huggingface_hub
!huggingface-cli download --resume-download --local-dir-use-symlinks False OpenGVLab/InternVL2-4B --local-dir InternVL2-4B


/content/drive/MyDrive/FYP/InternVL/internvl_chat/pretrained
/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 22 files:   0% 0/22 [00:00<?, ?it/s]Downloading 'configuration_intern_vit.py' to 'InternVL2-4B/.cache/huggingface/download/configuration_intern_vit.py.ac60112c79abc35627a5b6b58e760c2f78e71839.incomplete'

configuration_intern_vit.py: 100% 5.55k/5.55k [00:00<00:00, 25.3MB/s]
Download complete. Moving file to InternVL2-4B/configuration_intern_vit.py

configuration_internvl_chat.py: 100% 3.83k/3.83k [00:00<00:00, 14.4MB/s]
Download c

Mounted at /content/drive


In [3]:

# Code to load the model and test it out. This code is only for testing

import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([
        int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
        for idx in range(num_segments)
    ])
    return frame_indices

def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
        img = transform(img)
        pixel_values_list.append(img)
        num_patches_list.append(1)  # Since there's only one patch per frame in this setting

    pixel_values = torch.stack(pixel_values_list)
    return pixel_values, num_patches_list

# Load the pre-trained InternVL2-8B model
path = "/content/drive/MyDrive/FYP/InternVL/internvl_chat/pretrained/InternVL2-4B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

# Define inference parameters
generation_config = dict(max_new_tokens=1024, do_sample=False)

# Load video
video_path = '/content/drive/MyDrive/FYP/Dataset/Students Fighting /recording (1).mkv'  # Replace with your video file path
pixel_values, num_patches_list = load_video(video_path, num_segments=8, max_num=1)
pixel_values = pixel_values.to(torch.bfloat16).cuda()

# Construct the question and video prompt
video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])
question = video_prefix + "If this video shows students fighting, then output 1, otherwise output 0."

# Perform inference
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


Phi3ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


User: Frame1: <image>
Frame2: <image>
Frame3: <image>
Frame4: <image>
Frame5: <image>
Frame6: <image>
Frame7: <image>
Frame8: <image>
If this video shows students fighting, then output 1, otherwise output 0.
Assistant: 1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.24.0 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [2]:
# create an API endpoint (/predict) to access the model. Pass the Video and the prompt to the model

import os
import tempfile
from fastapi import FastAPI, UploadFile, Form
from transformers import AutoModel, AutoTokenizer
import torch
from decord import VideoReader, cpu
from torchvision import transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
import numpy as np

# Constants for preprocessing
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

# Define preprocessing functions
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([
        int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
        for idx in range(num_segments)
    ])
    return frame_indices

def load_video(video_path, bound=None, input_size=448, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list = []
    transform = build_transform(input_size=input_size)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
        img = transform(img)
        pixel_values_list.append(img)

    pixel_values = torch.stack(pixel_values_list)
    num_patches_list = [1] * len(frame_indices)
    return pixel_values, num_patches_list

# Initialize FastAPI
app = FastAPI()

# Load your model
  # Adjust this to your model's location
model_path = "/content/drive/MyDrive/FYP/InternVL/internvl_chat/pretrained/InternVL2-4B"

model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)

generation_config = dict(max_new_tokens=1024, do_sample=False)

@app.post("/predict")
async def predict(prompt: str = Form(...), video: UploadFile = Form(...)):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video:
        temp_video.write(await video.read())
        video_path = temp_video.name

    try:
        pixel_values, num_patches_list = load_video(video_path, num_segments=8)
        pixel_values = pixel_values.to(torch.bfloat16).cuda()

        video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])
        question = video_prefix + prompt

        response, history = model.chat(
            tokenizer,
            pixel_values,
            question,
            generation_config,
            num_patches_list=num_patches_list,
            history=None,
            return_history=True
        )

        return {"prompt": prompt, "response": response}
    finally:
        os.remove(video_path)


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


Phi3ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Used by me to test the app endpoint. This is not part of the FAST API App.

import nest_asyncio
from pyngrok import ngrok
import uvicorn

!ngrok config add-authtoken 2pEm9JyaOGl1P5Lg3b9wkBkEwAt_3gmxLW8kEKJxJiDfwv7T4


# Allow FastAPI to run in Colab's async environment
nest_asyncio.apply()

# Create a public URL with ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Start the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:     Started server process [512]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://b022-34-141-228-179.ngrok-free.app" -> "http://localhost:8000"
INFO:     2400:adc1:1bb:3900:acf7:2464:a61c:86a6:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2400:adc1:1bb:3900:acf7:2464:a61c:86a6:0 - "GET /openapi.json HTTP/1.1" 200 OK


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


INFO:     34.134.225.234:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [512]


In [4]:
# testing the endpoint by passing the prompt and the video.

import requests

url = "https://b022-34-141-228-179.ngrok-free.app/predict"
files = {
    "video": open("/content/drive/MyDrive/FYP/Dataset/Students Fighting /recording (1).mkv", "rb"),
}
data = {
    "prompt": "If this video shows students fighting, then output 1, otherwise output 0.",
}

response = requests.post(url, files=files, data=data)
print(response.json())


  Using cached python_multipart-0.0.17-py3-none-any.whl.metadata (1.8 kB)
Using cached python_multipart-0.0.17-py3-none-any.whl (24 kB)
